# Match stereo image pairs to labeled images and create csv reference file

In [5]:
###import libraries
import csv
from pathlib2 import Path

###global variables
str_labelFile = '../files/csv/biograd_calm_labeled.csv'
str_stereoFile = '../files/csv/biograd_calm_stereoimgs.csv'
selected_stereo_imgs = list()
stereo_csv_list = list()

###code

#go through stereo file, and create a list of the images selected for labeling
with open(str_stereoFile,'r') as stereofile:
    imageReader = csv.reader(stereofile, delimiter=',')
    for row in imageReader:
        str_img = row[3]
        image_path = Path(str_img)
        selected_stereo_imgs.append(image_path.name)
        stereo_csv_list.append(row)

#debug created list - comment if necessary
#print(len(selected_stereo_imgs))
#print(selected_stereo_imgs[0])
#print(selected_stereo_imgs)

In [10]:
###GLOBAL VARIABLES
str_finalFile = '../files/csv/biograd_calm_gestures.csv'
csv_class_column = 17
finalFile_fieldNames = ['stereo left','stereo right','selected img','label name', 'label id', 'roi params']
labels_dict = {"start_comm":0,"end_comm":1,"up":2,"down":3,"photo":4,"backwards":5,"carry":6,
              "boat":7,"here":8,"mosaic":9,"num_delimiter":10,"one":11,"two":12,"three":13,
               "four":14,"five":15,"other":-1}

#open output file to write when necessary
ofile = open(str_finalFile,'w')
writer_finalFile = csv.DictWriter(ofile,fieldnames=finalFile_fieldNames)
writer_finalFile.writeheader()

#open label file to backtrace the stereo images where it comes from
with open(str_labelFile, 'r') as csvfile:
    imageReader = csv.reader(csvfile, delimiter=',')
    flag_ignore_row = True
    for label_info in imageReader:
        #ignore 1st row containing column headers
        if  flag_ignore_row:
            flag_ignore_row = False
        else:
            str_img = label_info[0]
            image_path = Path(str_img[1:-1]) #this file has quotes around strings - remove them
            
            #search for this string in the list of selected stereo images
            for idx,img in enumerate(selected_stereo_imgs):
                if image_path.name == img:
                    #print("1: {}, 2: {}".format(image_path.name,stereo_csv_list[idx]))
                    stereo_img_info = stereo_csv_list[idx]
                    roi_info_idx = labels_dict[label_info[csv_class_column]]+1;
                    if roi_info_idx != 0:
                        writer_finalFile.writerow({'stereo left':stereo_img_info[0],
                                                  'stereo right':stereo_img_info[1],
                                                  'selected img':stereo_img_info[2],
                                                  'label name':label_info[csv_class_column],
                                                  'label id':labels_dict[label_info[csv_class_column]],
                                                  'roi params':label_info[roi_info_idx]})                    

                    break
                

ofile.close()